In [67]:
from cdp_backend.database import models as db_models
from cdp_backend.pipeline.transcript_model import Transcript
import fireo
from gcsfs import GCSFileSystem
from google.auth.credentials import AnonymousCredentials
from google.cloud.firestore import Client

import pickle

In [2]:
# Connect to the database
fireo.connection(client=Client(
    project="cdp-milwaukee-9f60e352",
    credentials=AnonymousCredentials()
))

# Connect to the file store
fs = GCSFileSystem(project="cdp-milwaukee-9f60e352", token="anon")

In [59]:
# get all transcripts
all_transcripts = list(db_models.Transcript.collection.fetch(0))
len(all_transcripts)

417

In [65]:
# sentences for all transcripts - runs in ~6 min on home wifi
transcripts = dict()
for transcript in all_transcripts:
    with fs.open(transcript.file_ref.get().uri, "r") as open_resource:
        transcripts[transcript.session_ref.get().id] = [
            sentence.text for sentence in Transcript.from_json(open_resource.read()).sentences]

In [69]:
with open("transcripts.pickle", "wb") as open_resource:
    pickle.dump(transcripts, open_resource)

In [ ]:
print(f"Number of transcripts: {len(transcripts)}")
print(f"Number of sentences: {sum([len(sentences) for sentences in transcripts.values()])}")
print(f"Number of words: {sum([sum([len(sentence.split()) for sentence in sentences]) for sentences in transcripts.values()])}")

Number of transcripts: 412
Number of sentencess: 328733
Number of words: 4843941
